In [2]:
import os
import numpy as np
import pandas as pd
import json

In [8]:

train_files = os.listdir('release-update-6-26-12')

In [9]:
train_files = train_files[:100]

In [10]:
dataset = []

In [16]:
def generate_train_data(folder_dir, n = 100):
    import xml.etree.ElementTree as ET

    files = os.listdir(folder_dir)
    files = files[:n]
    train_data = []
    for file in files:
        try:
            root = ET.parse(os.path.join(folder_dir, file)).getroot()
            out = [{field: tag.attrib[field] for field in ['text', 'val', 'type', 'mod', 'start']} for tag in root[1] if
                   tag.tag == 'TIMEX3']
            text = root[0].text
            for tag in out:
                tag['line'] = text.count('\n', 0, int(tag['start']))
                tag['text'] = tag['text'].replace('\n', ' ')
                tag['val'] = tag['val'].replace('\n', ' ')
                tag['file'] = file
            

            import re

            tags = [{'val': tag['val'], 'line': tag['line'], 'text': tag['text']} for tag in out if
                    tag['type'] == 'DATE' and bool(re.search(r'\d', tag['text']))]

            l = list(set([json.dumps(tag) for tag in tags]))
            tags = [json.loads(tag) for tag in l]

            train_data.append({'text': text, 'file': file, 'tags': tags})
        except:
            exc_info = sys.exc_info()
            print(exc_info)
            print(file)

    return train_data

train_chunks = generate_train_data('release-update-6-26-12/')

In [17]:
len(train_chunks)

100

In [18]:
for chunk in train_chunks:
    json.dump(chunk, open('json/' + chunk['file'][:-4] + '.json', 'w'))

In [5]:
files = os.listdir('i2b2-2012')
counts = []
for file in files:
    with open('i2b2-2012/' + file, 'r') as f:
        data = json.load(f)
        counts.append(len(data['annotations']))

sum(counts)/len(counts), max(counts), min(counts)

(5.485714285714286, 22, 2)